In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import sys
from catboost import CatBoostRegressor, Pool, MetricVisualizer
from sasaki_features import add_feature_position_month
from datetime import datetime

sys.path.append("../dora/models")
from utils import read_data, process_time, merge_data, promotionAggregation

## some functions from dora

### (por algum motivo eu nao consegui importar)

In [2]:
def promo_detector_fixed(orders, aggregation=True, mode=True):
    """
    This function adds a "promotion" column at "orders.csv".
    It verifies if an item of an order is being sold cheaper than it's prices "mode"/"mean". 
    Case affirmative, a '1' will be added in 'promotion' column in the line of the order.

    Parameters: orders -> Orders DataFrame
                aggregation -> Flag that mantains or not the "salesPriceMode" in our returned DataFrame
                True => Return will have the column
                mode -> Decision method flag (Default 'True'). If "True", the function will 
                use the 'mode' of the prices to decide if an item is being sold below it's normal price. 
                If 'False', we'll use the "mean" of the prices.
                
    Returns: our orders Dataframe with 2 new columns ("salesPriceMode" and "promotion")
    """
    
    new_df = pd.DataFrame()
      
    def agregationMode(x): return x.value_counts().index[0] if mode else 'mean'
    
    for i in range(13, -1, -1):
        # Getting an itemID / salesPriceMode Dataframe
        # salesPriceMode column will store the 
        # 'mean'/'mode' of our items
        current_agg = orders.loc[orders.group_backwards > i].groupby(['itemID']).agg(salesPriceMode=('salesPrice', agregationMode))
        
        current_agg['promotion'] = 0
        orders_copy = orders.loc[orders.group_backwards == i + 1].copy()
        
        current_orders_with_promotion = pd.merge(orders_copy, current_agg, how='inner', left_on='itemID', right_on='itemID')
        
        # For every item whose salesPrice is lower than the 'mean'/'mode',
        # we'll attribute 1 to it's position in 'promotion' column
        current_orders_with_promotion.loc[current_orders_with_promotion['salesPrice'] <
                                                       current_orders_with_promotion['salesPriceMode'], 'promotion'] = 1
        
        new_df = pd.concat([new_df, current_orders_with_promotion])
    
    
    week_13 = orders.loc[orders.group_backwards == 13].copy()
    week_13['salesPriceMode'] = 0
    week_13['promotion'] = 0
    
    new_df = pd.concat([new_df, week_13])
    
    if (not(aggregation)):
        new_df.drop(
            'salesPriceMode', axis=1, inplace=True)
        
    new_df.sort_values(by=['group_backwards', 'itemID'], inplace=True)
    
    return new_df

In [3]:
def baseline_score(prediction, target, simulatedPrice):
    prediction = prediction.astype(int)

    return np.sum((prediction - np.maximum(prediction - target, 0) * 1.6)  * simulatedPrice)

## importing orders and applying already made features

In [4]:
infos, items, orders = read_data("../main/datasets/")
process_time(orders)

orders_columns = set(orders.columns)
print(orders_columns)

{'salesPrice', 'days', 'days_backwards', 'week_backwards', 'group_backwards', 'time', 'order', 'transactID', 'itemID'}


In [5]:
orders2 = promo_detector_fixed(orders)
orders2 = promotionAggregation(orders2, items) #depois arrumar promotion agregation


orders2_columns = set(orders2.columns)
print(orders2_columns - orders_columns)

{'salesPrice_mean', 'category2', 'brand', 'promotion_mean', 'recommendedRetailPrice', 'category1', 'orderSum', 'category3', 'manufacturer', 'customerRating'}


## Adding zero ordemSum rows

In [6]:
valid_ids = set( orders['itemID'].unique() )
all_ids = set(items['itemID'].unique())

#OBS: if add_all_ids == false could cause dataleak, since you know the items in the table are sold
#at least one time

#add rows with zero values to the last timestamp (without dataleak)
def prepareTimeStamp(data, time, add_all_ids = True):
    #finding ids to add
    last_row = data.query(f"group_backwards == {time}")
    last_row_ids = set(last_row['itemID'].unique())
    
    if(not add_all_ids):
        news_ids = list(valid_ids - last_row_ids)
    else:
        news_ids = list(all_ids - last_row_ids)
    
    #adding new rows
    new_rows = pd.DataFrame({'itemID': news_ids })
    new_rows['group_backwards'] = time
    new_rows['salesPrice_mean'] = 0
    new_rows['orderSum'] = 0
    new_rows['promotion_mean'] = 0
    
    
    return data.append(new_rows)

In [7]:
def prepareOrders(data, add_all_ids = True):
    
    for time in data['group_backwards'].unique():
        data = prepareTimeStamp(data, time, add_all_ids)
        

    #remove overlapping column from merge with items
    overlapping = list(set(data.columns) and set(items.columns)  )
    overlapping.remove('itemID')
    data.drop(overlapping, axis=1, inplace=True)
    
    data = pd.merge(data,items, on=['itemID'], how="left", validate="m:1")
    return data



orders3 = orders2.copy()
orders3 = prepareOrders(orders3)
orders3 = orders3.sort_values(['group_backwards', 'itemID'], ascending=[False, True], ignore_index=True)

## New features 1

In [8]:
orders4 = add_feature_position_month(orders3)

In [9]:
#apply function without information from futures
def apply_to_serie(data, function,extraParans={}):
    
    new_data = pd.DataFrame()
 
    for time in data['group_backwards'].unique():
        new_rows = function(data,time,**extraParans)
        new_data = pd.concat([new_data, new_rows])
        

    return new_data

In [10]:
def add_feature_freq(data,time):
    orders_aux = orders.query(f"group_backwards >  {time}")#remember, its backwards
    
    nDays = orders_aux['days'].nunique()
    nWeek = orders_aux['week_backwards'].nunique()
    nGroup = orders_aux['group_backwards'].nunique()

    newInfo = items[['itemID']].copy()
    newInfo["group_backwards"] = time
    
    #how many days in average the item is sold in day/week/pair
    newInfo['freq_day'] = orders_aux.groupby('itemID', as_index=False)['days'].nunique()/nDays
    newInfo['freq_week'] = orders_aux.groupby('itemID', as_index=False)['week_backwards'].nunique()/nWeek
    newInfo['freq_group'] = orders_aux.groupby('itemID', as_index=False)['group_backwards'].nunique()/nGroup
    
    current_time = data.query(f"group_backwards == {time}")
    return pd.merge(current_time,newInfo, on=['itemID','group_backwards'], how="left", validate="m:1")

orders4 = apply_to_serie(orders4,  add_feature_freq)

In [11]:
#ordenando features categoricas usando como metrica 
#vendas totais medias entre itemID de mesma categoria
def add_feature_ord_cat(data, time, category):
    orders_aux = data.query(f"group_backwards > {time}")#remember, its backwards

    newInfo = orders_aux.groupby(category,as_index=False).agg({'orderSum' : ['sum'],'itemID' : ['count']})
    newInfo["group_backwards"] = time
    

    #calculando a metrica para cada item diferente da coluna
    newInfo[f'avg_sales_{category}'] = newInfo[( 'orderSum',   'sum')] / newInfo[( 'itemID', 'count')]
    newInfo = newInfo[[category,"group_backwards", f'avg_sales_{category}']]
    newInfo.columns = [category,"group_backwards", f'avg_sales_{category}']
    
    
    current_time = data.query(f"group_backwards == {time}")
    return pd.merge(current_time,newInfo, on=[category,'group_backwards'], how="left", validate="m:1")


orders4 = apply_to_serie(orders4,  add_feature_ord_cat, extraParans={"category": "category3"})
orders4 = apply_to_serie(orders4,  add_feature_ord_cat, extraParans={"category": "brand"})
orders4 = apply_to_serie(orders4,  add_feature_ord_cat, extraParans={"category": "manufacturer"})

In [12]:
def add_feature_min_max(data,time):
    orders_aux = data.query(f'group_backwards > {time}')

    newInfo = items[['itemID']].copy()
    newInfo['group_backwards'] = time
    
    #minimun and maximum sales in a pair
    #too much zeros, trying for last 4 pairs
    newInfo['min_sale'] = orders_aux.groupby(['itemID'])['orderSum'].min()
    newInfo['max_sale'] = orders_aux.groupby(['itemID'])['orderSum'].max()


    #minimun and maximum sales in a group recent
    order_recent = data.query(f'group_backwards > {time} & group_backwards < {time+4}')
    newInfo['min_sale_rec'] = order_recent.groupby(['itemID'])['orderSum'].min()
    newInfo['max_sale_rec'] = order_recent.groupby(['itemID'])['orderSum'].max()

    current_time = data.query(f"group_backwards == {time}")
    return pd.merge(current_time,newInfo, on=['itemID','group_backwards'], how="left", validate="m:1")

orders4 = apply_to_serie(orders4,  add_feature_min_max)

### add feature first appearance

In [13]:
#finding first apperance
not_zero_order = orders4.query('orderSum != 0')
first_appearance = not_zero_order.groupby('itemID',as_index=False)['group_backwards'].max()#remenber backwards
first_appearance.columns = ['itemID','first_appearance']

orders4 = pd.merge(orders4, first_appearance, on="itemID",how="left", validate="m:1")

#putting in relation with the current timestamp
#positive means that the itemID was never sold
#negative means that the itemID was already sold
orders4['first_appearance'] = orders4['group_backwards'] - orders4['first_appearance'] 

#removing dataleak
func = lambda x : np.nan if x >= 0 else x
orders4['first_appearance'] = orders4['first_appearance'].apply(func)


In [14]:
orders4_columns = set(orders4.columns)
print(orders4_columns - orders2_columns)

{'posM_f_group', 'min_sale_rec', 'max_sale_rec', 'freq_week', 'posM_m_group', 'posM_l_group', 'avg_sales_brand', 'first_appearance', 'freq_group', 'max_sale', 'avg_sales_category3', 'freq_day', 'avg_sales_manufacturer', 'min_sale'}


In [15]:
#freq feature has lots of nan; just checking if there is a error
print(len(first_appearance[first_appearance.first_appearance == 1]) * 13 + \
len(first_appearance[first_appearance.first_appearance == 2]) * 12 + \
len(first_appearance[first_appearance.first_appearance == 3]) * 11 + \
len(first_appearance[first_appearance.first_appearance == 4]) * 10 + \
len(first_appearance[first_appearance.first_appearance == 5]) * 9 + \
len(first_appearance[first_appearance.first_appearance == 6]) * 8 + \
len(first_appearance[first_appearance.first_appearance == 7]) * 7 + \
len(first_appearance[first_appearance.first_appearance == 8]) * 6 + \
len(first_appearance[first_appearance.first_appearance == 9]) * 5 + \
len(first_appearance[first_appearance.first_appearance == 10]) * 4 + \
len(first_appearance[first_appearance.first_appearance == 11]) * 3 + \
len(first_appearance[first_appearance.first_appearance == 12]) * 2 + \
len(first_appearance[first_appearance.first_appearance == 13]) * 1 + \
len(all_ids - valid_ids) * 13 )

display(orders4.freq_day.isnull().sum())

74609


74609

## Shift
### added the feature salesPrice_mean_ from older pairs

In [16]:
orders5 = orders4.copy()

for i in range(1, 3):
    # Carrying the data of weeks t-1
    orders5[f'orderSum_{i}'] = orders5.groupby('itemID')['orderSum'].shift(i)
    orders5[f'promotion_mean_{i}'] = orders5.groupby('itemID')['promotion_mean'].shift(i)
    orders5[f'salesPrice_mean_{i}'] = orders5.groupby('itemID')['salesPrice_mean'].shift(i)
    
    # Getting the difference of the orders and promotions between weeks t-1 and t-2...
    orders5[f'orderSum_diff_{i}'] = orders5.groupby('itemID')[f'orderSum_{i}'].diff()
    orders5[f'promotion_mean_diff_{i}'] = orders5.groupby('itemID')[f'promotion_mean_{i}'].diff()
    orders5[f'salesPrice_mean_diff_{i}'] = orders5.groupby('itemID')[f'salesPrice_mean_{i}'].diff()

orders5 =orders5.fillna(np.inf)

orders5_columns = set(orders5.columns)
print(orders5_columns - orders4_columns)

{'promotion_mean_2', 'salesPrice_mean_diff_2', 'salesPrice_mean_1', 'orderSum_2', 'orderSum_1', 'salesPrice_mean_diff_1', 'orderSum_diff_2', 'orderSum_diff_1', 'promotion_mean_1', 'salesPrice_mean_2', 'promotion_mean_diff_1', 'promotion_mean_diff_2'}


## Custom metrics

In [17]:
class custom_obj(object):
    def __iter__(self):
        return iter('custom')
    
    def get_final_error(self, error, weight):
    
        return error

    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weight):
        approx = approxes[0]

        error_sum = 0.0
        weight_sum = 0.0

        for prediction,t,w in zip(approx, target, weight):
            
            weight_sum += w
            
            error_sum += -1* (prediction - (np.maximum(prediction - t, 0) * 1.6))  * w

        return error_sum, weight_sum
    def calc_ders_range(self, approxes, targets, weights):
        pred = np.array(approxes)
        target = np.array(targets)
        weight = np.array(weights)
        
        
        der1 = -2 *weight* (pred - (np.maximum(pred - target, 0) * 1.6)) * (1 - (pred > target) * 1.6)
        der2 = -2 *weight* (1 - (pred > target) * 1.6) ** 2

        return list(zip(der1,der2))

In [18]:
orders6 = orders5.copy()

display(orders6.columns)

Index(['group_backwards', 'itemID', 'orderSum', 'promotion_mean',
       'salesPrice_mean', 'brand', 'manufacturer', 'customerRating',
       'category1', 'category2', 'category3', 'recommendedRetailPrice',
       'posM_f_group', 'posM_m_group', 'posM_l_group', 'freq_day', 'freq_week',
       'freq_group', 'avg_sales_category3', 'avg_sales_brand',
       'avg_sales_manufacturer', 'min_sale', 'max_sale', 'min_sale_rec',
       'max_sale_rec', 'first_appearance', 'orderSum_1', 'promotion_mean_1',
       'salesPrice_mean_1', 'orderSum_diff_1', 'promotion_mean_diff_1',
       'salesPrice_mean_diff_1', 'orderSum_2', 'promotion_mean_2',
       'salesPrice_mean_2', 'orderSum_diff_2', 'promotion_mean_diff_2',
       'salesPrice_mean_diff_2'],
      dtype='object')

In [19]:
#categorical features
cat_features = ['brand','manufacturer','category1','category2','category3']

#passing to integer
for f in cat_features:
    orders6[f] = orders6[f].map(lambda x : int(x))
    
weight =pd.merge(orders6, infos[["itemID", "simulationPrice"]], 
                     on="itemID", validate="m:1")
weightt = weight[["itemID","group_backwards","simulationPrice"]]

params = {'iterations': 200, 
         'loss_function':'RMSE',
         'use_best_model': True,
         'early_stopping_rounds': 30,
}

params2= {'loss_function':custom_obj(),
         'iterations': 200, 
         'eval_metric':custom_obj(),
         'use_best_model': True,
         'early_stopping_rounds': 30,
         'subsample':1,
         }


params3= {'loss_function':'RMSE',
         'iterations': 200, 
         'eval_metric':custom_obj(),
         'early_stopping_rounds': 30,
         'use_best_model': True,
         }



In [20]:
#ADD NOT SOLD ITEMS IF YOUR MODEL DONT PREDICT ALL
def get_pred(modelo,test, nome):
    
    
    test_pool = Pool(test.drop(columns=["orderSum"]),
                 weight= test['salesPrice_mean'],
                 cat_features= cat_features
    ) 
        
    preds = modelo.predict(test_pool)

    #all prediction need to be positive and integer
    sold_items = test.copy()
    preds = [max(x,0) for x in preds ]
    sold_items['demandPrediction'] = preds
    sold_items = sold_items[["itemID", "demandPrediction"]]

    sold_items["demandPrediction"] = sold_items["demandPrediction"].astype(np.uint8)

    #to kagle csv
    return sold_items.sort_values(['itemID'],  ignore_index=True)
    #final.to_csv(f"pred/{nome}.csv", index=False, sep='|')

In [21]:
def get_result(data, not_drop_columns):
    
    new_features = list(orders5_columns - orders2_columns)
    for colum in not_drop_columns :
        new_features.remove(colum)
    
    data = data.drop(columns=new_features)
    
    test = data.query('group_backwards == 1')
    val = data.query('group_backwards == 2')
    train = data.query('group_backwards >= 3')


    train_pool = Pool(
        data= train.drop(columns=["orderSum"]), 
        label= train['orderSum'], 
        weight= weightt.query('group_backwards >= 3') ,
        cat_features= cat_features
    )
    
    validation_pool = Pool(
        data= val.drop(columns=["orderSum"]), 
        label= val['orderSum'], 
        weight= weightt.query('group_backwards == 2'),
        cat_features= cat_features
    )
    
    
    
    model=CatBoostRegressor(**params) 
    model.fit(train_pool,eval_set=validation_pool , verbose=False)
    
    #model2=CatBoostRegressor(**params2) 
    #model2.fit(train_pool,eval_set=validation_pool , verbose=False)
    
    model3=CatBoostRegressor(**params3) 
    model3.fit(train_pool,eval_set=validation_pool , verbose=False)
    
    
    target = test['orderSum'].values
    predct1 =get_pred(model,test, 'cat_pos1')['demandPrediction'].values
    predct3 =get_pred(model3,test, 'cat_pos1')['demandPrediction'].values
    
    score1 = baseline_score(predct1, target, infos['simulationPrice'])
    score3 = baseline_score(predct3, target, infos['simulationPrice'])
    
    return score1, score3


In [22]:
print(orders5_columns - orders2_columns)

{'posM_f_group', 'promotion_mean_2', 'posM_m_group', 'orderSum_2', 'orderSum_1', 'freq_group', 'promotion_mean_1', 'max_sale_rec', 'posM_l_group', 'salesPrice_mean_diff_2', 'avg_sales_brand', 'orderSum_diff_1', 'salesPrice_mean_2', 'promotion_mean_diff_1', 'min_sale', 'salesPrice_mean_1', 'salesPrice_mean_diff_1', 'orderSum_diff_2', 'first_appearance', 'max_sale', 'avg_sales_category3', 'freq_day', 'avg_sales_manufacturer', 'min_sale_rec', 'freq_week', 'promotion_mean_diff_2'}


## results

In [23]:
print(get_result(orders6,[]))#all new features

print(get_result(orders6, list(orders5_columns - orders2_columns)))#no new features

print(get_result(orders6, ['posM_f_group', 'posM_m_group','posM_l_group']))#position in month



(357700.09799999953, 367224.22599999956)
(-190105.47200000065, -1001292.8480000008)
(609259.2739999995, -5938897.162000001)


In [24]:
print(get_result(orders6, ['first_appearance']))#'first_appearance'

print(get_result(orders6, ['freq_day','freq_group','freq_week']))#frequency

print(get_result(orders6, ['avg_sales_brand','avg_sales_manufacturer','avg_sales_category3']))#avg sales

print(get_result(orders6, ['min_sale','max_sale','max_sale_rec','min_sale_rec']))#min max

print(get_result(orders6, ['max_sale_rec','min_sale_rec']))#min max recent

(562592.4199999996, 380524.3699999995)
(-238435.58400000064, 115259.5059999995)
(-726034.5860000008, -864571.0920000008)
(114680.34999999955, 183382.8399999995)
(292124.8939999995, -27755.78200000053)


In [25]:
print(get_result(orders6, ['salesPrice_mean_1','salesPrice_mean_diff_1','salesPrice_mean_2','salesPrice_mean_diff_2']))

print(get_result(orders6, ['promotion_mean_1','promotion_mean_2','promotion_mean_diff_1','promotion_mean_diff_2']))

print(get_result(orders6, ['orderSum_1','orderSum_2','orderSum_diff_1','orderSum_diff_2']))

(356447.6979999995, 295010.98399999953)
(-11699.508000000511, -48602.39800000058)
(134665.21399999945, 341139.3339999995)


In [26]:
print(get_result(orders6, ['salesPrice_mean_1','salesPrice_mean_diff_1','salesPrice_mean_2','salesPrice_mean_diff_2',
                          'first_appearance','min_sale','max_sale','max_sale_rec','min_sale_rec',
                          'orderSum_1','orderSum_2','orderSum_diff_1','orderSum_diff_2']))

(327392.9499999995, -12338.390000000538)
